## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [5]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    'Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
    'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked',
    'PercentSalaryHike', 'TotalWorkingYears', 'WorkLifeBalance'
]

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes


Age                        int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
JobInvolvement             int64
JobLevel                   int64
JobSatisfaction            int64
NumCompaniesWorked         int64
PercentSalaryHike          int64
TotalWorkingYears          int64
WorkLifeBalance            int64
dtype: object

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_df = X_df.apply(pd.to_numeric, errors='coerce')

In [8]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)


In [46]:
# Create a OneHotEncoder for the Department column
encoder_department = OneHotEncoder(sparse_output=False,handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
encoder_department.fit(np.array(y_train['Department']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = encoder_department.transform(np.array(y_train['Department']).reshape(-1, 1))
y_test_department = encoder_department.transform(np.array(y_test['Department']).reshape(-1, 1))

In [48]:
# Create a OneHotEncoder for the Attrition column
encoder_attrition = OneHotEncoder(sparse_output=False,handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
encoder_attrition.fit(np.array(y_train['Attrition']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = encoder_attrition.transform(np.array(y_train['Attrition']).reshape(-1, 1))
y_test_attrition = encoder_attrition.transform(np.array(y_test['Attrition']).reshape(-1, 1))


## Create, Compile, and Train the Model

In [56]:
# Find the number of columns in the X training data
num_features = len(X_train.columns)

# Create the input layer
inputs = layers.Input(shape=(num_features,), name='inputs')

# Create at least two shared layers
shared_layer_1 = layers.Dense(64, activation='relu', name='shared_layer_1')(inputs)
shared_layer_2 = layers.Dense(128, activation='relu', name='shared_layer_2')(shared_layer_1)

In [58]:
# Create a branch for Department # with a hidden layer and an output layer
# Create the hidden layer
department_branch = layers.Dense(32, activation='relu', name='department_branch')(shared_layer_2)

# Create the output layer
department_output = layers.Dense(y_train_department.shape[1], activation='sigmoid', name='department_output')(department_branch)



In [60]:
# Create a branch for Attrition # with a hidden layer and an output layer
# Create the hidden layer
attrition_branch = layers.Dense(32, activation='relu', name='attrition_branch')(shared_layer_2)

# Create the output layer
attrition_output = layers.Dense(y_train_attrition.shape[1], activation='sigmoid', name='attrition_output')(attrition_branch)


In [62]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

In [64]:
# Create the model
model = Model(inputs=inputs, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={
                  'department_output': 'categorical_crossentropy',
                  'attrition_output': 'binary_crossentropy'
              },
              metrics={
                  'department_output': 'accuracy',
                  'attrition_output': 'accuracy'
              })
# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)           │ (None, 11)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ shared_layer_1 (Dense)        │ (None, 64)                │             768 │ inputs[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ shared_layer_2 (Dense)        │ (None, 128)               │           8,320 │ shared_layer_1[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ department_branch (Dense)     │ (None, 32)                │           4,128 │ shared_layer_2[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attrition_branch (Dense)      │ (None, 32)                │           4,128 │ shared_layer_2[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ department_output (Dense)     │ (None, 3)                 │              99 │ department_branch[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attrition_output (Dense)      │ (None, 2)                 │              66 │ attrition_branch[0][0]     │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 17,509 (68.39 KB)

 Trainable params: 17,509 (68.39 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
# Train the model
fit_model = model.fit(scaled_X_train, {'department_output': y_train_department, 'attrition_output': y_train_attrition}, epochs=100)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - attrition_output_accuracy: 0.7506 - department_output_accuracy: 0.5536 - loss: 1.4719
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8462 - department_output_accuracy: 0.6504 - loss: 1.1795
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8655 - department_output_accuracy: 0.6626 - loss: 1.1115
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8440 - department_output_accuracy: 0.6506 - loss: 1.1252
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8447 - department_output_accuracy: 0.6602 - loss: 1.1376
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8522 - department_output_accuracy: 0.6821 - loss: 1.0601
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8413 - department_output_accuracy: 0.6689 - loss: 1.0759
Epoch 8/100
35/35 ━━━━━━━━━

In [70]:
# Evaluate the model with the testing data
# model_loss, department_accuracy, attrition_accuracy = model.evaluate(X_test, [y_test_department, y_test_attrition])
# Evaluate the model with the testing data
test_results = model.evaluate(scaled_X_test, {'department_output': y_test_department, 'attrition_output': y_test_attrition})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8375 - department_output_accuracy: 0.6193 - loss: 3.8691  


[4.554985046386719, 0.801630437374115, 0.5978260636329651]

In [88]:
# Print the accuracy for both department and attrition
print(f'Department Accuracy: {test_results[2]}')
print(f'Attrition Accuracy: {test_results[1]}')

Department Accuracy: 0.5978260636329651
Attrition Accuracy: 0.801630437374115


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?


```

YOUR ANSWERS HERE

1. Accuracy might not be the best metric to use on this data, especially if there is class imbalance between the categories in the Department and Attrition columns. Alternative metrics such as precision, recall, or F1-score could provide additional insights into the model's performance.

2. I used the 'sigmoid' activation function for both output layers, as it is suitable for multi-class classification tasks. It allows the model to output a probability distribution over the predicted classes for both the Department and Attrition targets.

3. The model could be improved by adjusting the architecture, such as adding more hidden layers or increasing the number of neurons. Additionally, hyperparameter tuning, using different activation functions, and experimenting with different optimizers could enhance performance. Collecting more data or using techniques to handle class imbalance might also be beneficial. Model can only be as good as its data. 